# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import random

In [ ]:
#list of tickers to map
#ticker_list = ["NVDA","TSM","SHEL","COP"]
#ticker_list = ["AAPL", "NVDA", "GOOG", "MSFT","TSM","BA","ORCL","HCTI","PL","DAL","AAL","SHEL","COP","GME"]
ticker_list=["AAPL","MSFT","GOOGL","AMZN","NVDA","TSLA","META","TSM","NFLX","AVGO","AMD","QCOM","INTC","IBM","ORCL","CRM","ADBE","SAP","NOW","SHOP","JPM","BAC","WFC","C","GS","MS","SCHW","BLK","AXP","UNH","JNJ","PFE","MRK","LLY","ABBV","BMY","VRTX","REGN","GILD","KO","PEP","MCD","SBUX","YUM","WMT","TGT","COST","HD","LOW","PG","CL","KMB","UL","PM","MO","EL","LVMUY","NKE","LULU","XOM","CVX","COP","SHEL","BP","EOG","SLB","HAL","BKR","NEE","DUK","SO","D","AEP","EXC","ED","PCG","SRE","PEG","BA","LMT","NOC","RTX","GD","TDG","HON","GE","ETN","EMR","CSCO","PANW","FTNT","ZS","CRWD","DDOG","SNOW","MDB","PLTR","ZS","RCL","CCL","DAL","AAL","UAL","LYV","BKNG","ABNB","UBER","LYFT"]

#list dates to map for
start_date = "2010-01-01"
end_date = "2025-06-12"

In [ ]:
plt.figure(figsize=(30,10))
all_dict = {}

for ticker in ticker_list:
    try:
        ticker_data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)
    except:
        print("No data for ticker " + ticker)
    plt.plot(ticker_data.index, ticker_data['Close'], label=f"Close_{ticker}")
    plt.plot(ticker_data.index, ticker_data['Open'], label=f"Open_{ticker}")
    #plt.plot(ticker_data.index, ticker_data['High'], label=f"High_{ticker}")
    #plt.plot(ticker_data.index, ticker_data['Low'], label=f"Low_{ticker}")

    all_dict[ticker] = ticker_data
#plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(30,10))
df_returns = pd.DataFrame()
for name in all_dict:
    returns = np.log(all_dict[name]['Close'] / all_dict[name]['Close'].shift(1))
    plt.plot(returns, label=name)
    df_returns[name]=returns
#for name in all_df:
#    returns = np.log((all_df[name]['Open'] / all_df[name]['Open'].shift(1)))+0.5
#    plt.plot(returns, label=name)

#plt.legend()
plt.grid(True)
plt.show()

In [ ]:
correlation_matrix = (round(df_returns.corr(),5))
print(correlation_matrix)



In [ ]:
plt.figure(figsize=(round(len(correlation_matrix)/4), round(len(correlation_matrix)/4)))
sns.heatmap(correlation_matrix, annot=False,cmap='RdYlBu_r', center=0, square=True, linewidths=0.1)
plt.title('Stock Correlation Matrix')
plt.show()

In [ ]:
# prints out each symbol and what has a correlation of {min_corr} or greater with it
min_corr = 70
for name in correlation_matrix.columns:
    corr_list = []
    for other_name in correlation_matrix.columns:
        if name != other_name:  # Don't include self-correlation
            if correlation_matrix.loc[name, other_name] >= min_corr:
                corr_list.append(other_name)
    if len(corr_list)>0:
        print(f"{name}: {corr_list}")

In [ ]:
def calculate_hedge_score(correlation_matrix, portfolio_weights=None):
    """
    Calculate a simple hedging score for a portfolio.

    Basic concept: A well-hedged portfolio has stocks that don't all move together.
    - High correlations = poor hedging (stocks move together)
    - Low/negative correlations = good hedging (stocks offset each other)

    Parameters:
    -----------
    correlation_matrix : pd.DataFrame
        Correlation matrix of stock returns (from your existing code)
    portfolio_weights : dict, optional
        Dictionary like {'AAPL': 0.3, 'NVDA': 0.2, ...}
        If None, assumes equal weights for all stocks

    Returns:
    --------
    dict : Simple hedging metrics
    """

    # If no weights provided, use equal weights
    if portfolio_weights is None:
        stocks = correlation_matrix.columns.tolist()
        portfolio_weights = {stock: round(1.0/len(stocks),3) for stock in stocks}

    # Make sure we only use stocks that exist in both
    stocks_in_portfolio = [stock for stock in portfolio_weights.keys()
                          if stock in correlation_matrix.columns]

    total_correlation = 0
    pair_count = 0

    # Look at all pairs of stocks in the portfolio
    for i, stock1 in enumerate(stocks_in_portfolio):
        for j, stock2 in enumerate(stocks_in_portfolio):
            if i < j:  # Don't double-count pairs
                # Get correlation between these two stocks
                correlation = correlation_matrix.loc[stock1, stock2]

                # Weight this correlation by how much of each stock we own
                weight1 = portfolio_weights[stock1]
                weight2 = portfolio_weights[stock2]
                weighted_correlation = correlation * weight1 * weight2

                total_correlation += weighted_correlation
                pair_count += weight1 * weight2

    # Average correlation (weighted by portfolio positions)
    if pair_count > 0:
        average_correlation = total_correlation / pair_count
    else:
        average_correlation = 0

    # Convert to hedge score (0-100 scale)
    # Higher score = better hedged
    # Logic: Low correlation = good hedging
    hedge_score = (1 - average_correlation) * 50 + 50

    # Simple interpretation
    if hedge_score >= 80:
        interpretation = "Excellent hedging - stocks move independently"
    elif hedge_score >= 60:
        interpretation = "Good hedging - reasonable diversification"
    elif hedge_score >= 40:
        interpretation = "Moderate hedging - some diversification"
    elif hedge_score >= 20:
        interpretation = "Poor hedging - stocks tend to move together"
    else:
        interpretation = "Very poor hedging - highly correlated portfolio"

    return {
        'hedge_score': round(hedge_score, 1),
        'average_correlation': round(average_correlation, 3),
        'interpretation': interpretation,
        'portfolio_size': len(stocks_in_portfolio)
    }

In [ ]:
# Evaluate default equal weights as well
calculate_hedge_score(correlation_matrix, portfolio_weights=None)

# Function to generate random portfolio weights
def random_portfolio_weights(symbols):
    weights = np.random.dirichlet(np.ones(len(symbols)), size=1)[0]
    return {symbols[i]: float(round(weights[i], 3)) for i in range(len(symbols))}

# Run 100 simulations
hedge_scores = []

for _ in range(100):
    random_weights = random_portfolio_weights(correlation_matrix.index)
    result = calculate_hedge_score(correlation_matrix, portfolio_weights=random_weights)
    hedge_scores.append({'weights': random_weights, 'score': result['hedge_score']})

best_portfolio = max(hedge_scores, key=lambda x: x['score'])
print("Best Hedge Score:", best_portfolio['score'])
print("Portfolio Weights:", best_portfolio['weights'])